- https://ckdals29672.tistory.com/m/26

In [ ]:
import numpy as np 
import os
import cv2
import seaborn as sns

import tensorflow as tf
from keras import models, layers
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [ ]:
# tf.config.list_physical_devices('GPU')

### 데이터셋 불러오기
- stanford_dogs 데이터셋 사용
- stanford_dogs 데이터셋에는 120개 견종의 이미지가 포함
- 총 20,580장의 이미지에서 12,000장은 학습셋, 나머지 8,580장은 평가용 데이터셋

In [ ]:
path = "/Users/yoonjeongyang/Desktop/AI-exer/augmentation/stanford_dogs/Images/"
dogs_name_list = os.listdir(path)

In [ ]:
# #### 각 이미지 디렉토리 이름 변경 ###########################################################
# for folder in dogs_name_list:
#     dir_name = folder
#     rename = folder.split("-")[1]
#     os.rename(path+dir_name, path+rename)
# dogs_name_list = os.listdir(path)

In [ ]:
#### 이미지 제너레이터 생성 #################################################################
datagen = ImageDataGenerator(
    zca_epsilon=1e-06,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.5, 1.5],
    shear_range=0.05,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    dtype='float32',
    validation_split=0.1,# set validation split
)

In [ ]:
img_size = (224,224)
batch_size = 10

In [ ]:
#### Train 이미지 제너레이터 생성 ##############################################################
train_generator = datagen.flow_from_directory(
    directory=path,
    target_size=img_size,
    batch_size= batch_size,
    subset='training'
)

In [ ]:
#### Valid 이미지 제너레이터 생성 #############################################################
valid_generator = datagen.flow_from_directory(
    directory=path,
    target_size=img_size,
    batch_size= batch_size,
    subset='validation'
)

In [ ]:
train_img = next(iter(train_generator))
print(train_img[0].shape)
print(train_img[1].shape)

In [ ]:
dogs_name_dict = {}
for key, name in enumerate(train_generator.class_indices):
    dogs_name_dict[key] = name

plt.figure(figsize=(10, 5))
for i in range(8):
    plt.subplot(2,4,i+1)
    num = np.random.randint(0,batch_size)
    plt.imshow((train_img[0][num]).astype("uint8"))
    plt.title(dogs_name_dict[np.argmax((train_img[1][num]))])
    plt.axis("off")
plt.show()

In [ ]:
#### 데이터 전처리 ################################################################################
## Batch크기 데이터셋의 값 분포(히스토그램)

plt.hist(x=train_img[0].reshape(-1,),bins=50)
plt.show()

In [ ]:
## 한개 이미지의 값 분포(히스토그램)
num = np.random.randint(0,batch_size)

plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.imshow((train_img[0][num]).astype("uint8"))
plt.title(dogs_name_dict[np.argmax((train_img[1][num]))])
plt.axis('off')
plt.subplot(1,2,2)
sns.histplot(x=train_img[0][num].reshape(-1,),bins=50,kde=True)

plt.show()

In [ ]:
## 한개 이미지의 값 -1~1로 정규화
img_normalize = (train_img[0][num] / 127.5) -1

plt.figure(figsize=(10,2))
for i in range(3):
    plt.subplot(1,6,2*i+1)
    plt.imshow(img_normalize[:,:,i])
    plt.title(dogs_name_dict[np.argmax((train_img[1][num]))]+'filter'+str(i))
    plt.axis('off')
    plt.subplot(1,6,2*i+2)
    sns.histplot(x=img_normalize[:,:,i].reshape(-1,),bins=50,kde=True)
    plt.axis('off')

plt.show()

In [ ]:
#### 정규화된 서로 다른 이미지 그래프 비교 #######################################################
## 이미지가 밝을 수록 골고루 퍼져있고 반면 이미지가 어두울 수록 -1.0쪽으로 쏠림

plt.figure(figsize=(10,5))
for i in range(2):
    num = np.random.randint(0,batch_size)
    img_normalize = (train_img[0][num] / 127.5) -1
    
    plt.subplot(2,2,2*i+1)
    plt.imshow(img_normalize[:,:,2], vmax=1)
    plt.title(dogs_name_dict[np.argmax((train_img[1][num]))])
    plt.colorbar()
    plt.axis('off')
    
    plt.subplot(2,2,2*i+2)
    sns.histplot(x=img_normalize[:,:,2].reshape(-1,),bins=50,kde=True)

plt.show()

<img src = 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdvWE8w%2FbtrFvrDVlfS%2FpVnkpzKqF9oLMkFMW7ewQ0%2Fimg.jpg'>

In [ ]:
#### 모델 만들기 ################################################################################
## inception_256 Model
def inception_256(input_image):
    conv1 = layers.Conv2D(64,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    
    conv3 = layers.Conv2D(96,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv3 = layers.Conv2D(128,(3,3),kernel_initializer="he_normal",padding="same")(conv3)
    
    conv5 = layers.Conv2D(16,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv5 = layers.Conv2D(32,(5,5),kernel_initializer="he_normal",padding="same")(conv5)

    convMax = layers.MaxPool2D(pool_size=(3,3),strides=1,padding="same")(input_image)
    convMax = layers.Conv2D(32,(1,1),kernel_initializer="he_normal",padding="same")(convMax)

    convCat = layers.Concatenate(axis=3)([conv1,conv3,conv5,convMax])
    return convCat

In [ ]:
## inception_384 Model
def inception_384(input_image):
    conv1 = layers.Conv2D(96,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    
    conv3 = layers.Conv2D(96,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv3 = layers.Conv2D(192,(3,3),kernel_initializer="he_normal",padding="same")(conv3)
    
    conv5 = layers.Conv2D(24,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv5 = layers.Conv2D(48,(5,5),kernel_initializer="he_normal",padding="same")(conv5)

    convMax = layers.MaxPool2D(pool_size=(3,3),strides=1,padding="same")(input_image)
    convMax = layers.Conv2D(48,(1,1),kernel_initializer="he_normal",padding="same")(convMax)

    convCat = layers.Concatenate(axis=3)([conv1,conv3,conv5,convMax])
    return convCat

In [ ]:
## Inception
def inception(input_image):
    conv1 = layers.Conv2D(128,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    
    conv3 = layers.Conv2D(128,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv3 = layers.Conv2D(256,(3,3),kernel_initializer="he_normal",padding="same")(conv3)
    
    conv5 = layers.Conv2D(24,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv5 = layers.Conv2D(64,(5,5),kernel_initializer="he_normal",padding="same")(conv5)

    convMax = layers.MaxPool2D(pool_size=(3,3),strides=1,padding="same")(input_image)
    convMax = layers.Conv2D(64,(1,1),kernel_initializer="he_normal",padding="same")(convMax)

    convCat = layers.Concatenate(axis=3)([conv1,conv3,conv5,convMax])
    return convCat

In [ ]:
## inception_720
def inception_720(input_image):
    conv1 = layers.Conv2D(180,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    
    conv3 = layers.Conv2D(180,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv3 = layers.Conv2D(360,(3,3),kernel_initializer="he_normal",padding="same")(conv3)
    
    conv5 = layers.Conv2D(45,(1,1),kernel_initializer="he_normal",padding="same")(input_image)
    conv5 = layers.Conv2D(90,(5,5),kernel_initializer="he_normal",padding="same")(conv5)

    convMax = layers.MaxPool2D(pool_size=(3,3),strides=1,padding="same")(input_image)
    convMax = layers.Conv2D(90,(1,1),kernel_initializer="he_normal",padding="same")(convMax)

    convCat = layers.Concatenate(axis=3)([conv1,conv3,conv5,convMax])
    return convCat

In [ ]:
input_image = layers.Input(shape=(224,224,3))

# Feature Extraction
layer0 = layers.Rescaling(scale=1./127.5, offset=-1)(input_image)

layer1 = layers.Conv2D(20,(3,3),kernel_initializer='he_normal',
                       kernel_regularizer=regularizers.l2())(layer0)
layer1 = layers.BatchNormalization()(layer1)  
layer1 = layers.Activation('relu')(layer1)                
layer1 = layers.MaxPool2D((2,2))(layer1)


layer2 = layers.Conv2D(64,(2,2),kernel_initializer='he_normal',
                       kernel_regularizer=regularizers.l2())(layer1)
layer2 = layers.BatchNormalization()(layer2)  
layer2 = layers.Activation('relu')(layer2)
layer2 = layers.MaxPool2D((2,2))(layer2)

layer3 = layers.Conv2D(192,(3,3),kernel_initializer='he_normal',
                       kernel_regularizer=regularizers.l2())(layer2)
layer3 = layers.BatchNormalization()(layer3)  
layer3 = layers.Activation('relu')(layer3)
layer3 = layers.MaxPool2D((2,2))(layer3)


# Inception Network
layer4 = inception_256(layer3)
layer4 = layers.BatchNormalization()(layer4)
layer4 = layers.Activation('relu')(layer4)

layer5 = inception_384(layer4)
layer5 = layers.BatchNormalization()(layer5)
layer5 = layers.Activation('relu')(layer5)

layer6 = inception(layer5)
layer6 = layers.BatchNormalization()(layer6)
layer7 = layers.Activation('relu')(layer6)

layer8 = inception(layer7)
layer8 = layers.BatchNormalization()(layer8)
layer8 = layers.Activation('relu')(layer8)

layer9 = inception(layer8)
layer9 = layers.BatchNormalization()(layer9)
layer9 = layers.Activation('relu')(layer9)
layer9 = layers.MaxPool2D((2,2))(layer9)


layer10 = inception_720(layer9)
layer10 = layers.BatchNormalization()(layer10)
layer10 = layers.Activation('relu')(layer10)

layer11 = inception_720(layer10)
layer11 = layers.BatchNormalization()(layer11)
layer11 = layers.Activation('relu')(layer11)

layer12 = inception_720(layer11)
layer12 = layers.BatchNormalization()(layer12)
layer12 = layers.Activation('relu')(layer12)
layer13 = layers.GlobalAveragePooling2D()(layer12)


# Fully Connected Network
layer14 = layers.Dense(512,kernel_initializer='he_normal',
                       kernel_regularizer=regularizers.l2())(layer13)
layer14 = layers.BatchNormalization()(layer14)
layer14 = layers.Activation('relu')(layer14)
layer14 = layers.Dropout(0.3)(layer14)

layer15 = layers.Dense(256,kernel_initializer='he_normal',
                       kernel_regularizer=regularizers.l2())(layer14)
layer15 = layers.BatchNormalization()(layer15)
layer15 = layers.Activation('relu')(layer15)
layer15 = layers.Dropout(0.2)(layer15)

layer16 = layers.Dense(256,kernel_initializer='he_normal',
                       kernel_regularizer=regularizers.l2())(layer15)
layer16 = layers.BatchNormalization()(layer16)
layer16 = layers.Activation('relu')(layer16)
layer16 = layers.Dropout(0.1)(layer16)

layer17 = layers.Dense(120,activation='softmax')(layer16)

In [ ]:
model = models.Model(inputs = input_image,outputs=layer17)
model.summary()

In [ ]:
# tf.keras.utils.plot_model(model,show_shapes=True)

In [ ]:
#### GPU 확인 ###########################################################################
! nvidia-smi

In [ ]:
#### 학습 설정 ######################################################################
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
#### 7. Model Checkpoint Callback ############################################################################
checkpt_dir = '/Users/yoonjeongyang/Desktop/AI-exer/augmentation/networks/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpt_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 save_freq='epoch',
                                                 verbose=1)

In [ ]:
#### 학습 실행 #####################################################################

history = model.fit(train_generator,epochs=200,validation_data=valid_generator)

In [ ]:
hist = history.history

In [ ]:
#### 학습 결과 그래프로 표현 ###########################################################
plt.plot(hist['loss'],label='loss')
plt.plot(hist['val_loss'],label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(hist['accuracy'],label='accuracy')
plt.plot(hist['val_accuracy'],label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#### 임의 배치 valid 데이터로 평가 및 예측 확인 #############################################
valid_img = next(iter(valid_generator))
model.evaluate(valid_img[0],valid_img[1])

In [ ]:
num = np.random.randint(0,batch_size)
plt.imshow(valid_img[0][num].astype('uint8'))
plt.title(dogs_name_dict[np.argmax(valid_img[1][num])])
plt.show()

y_pred = model(np.expand_dims(valid_img[0][num],axis=0)).numpy()[0]
names = np.argsort(y_pred)[::-1][:4]

for name in names:
    print(y_pred[name],":",dogs_name_dict[name])


In [ ]:
num = np.random.randint(0,batch_size)
plt.imshow(valid_img[0][num].astype('uint8'))
plt.title(dogs_name_dict[np.argmax(valid_img[1][num])])
plt.show()

y_pred = model(np.expand_dims(valid_img[0][num],axis=0)).numpy()[0]
names = np.argsort(y_pred)[::-1][:4]

for name in names:
    print(y_pred[name],":",dogs_name_dict[name])

In [ ]:
num = np.random.randint(0,batch_size)
plt.imshow(valid_img[0][num].astype('uint8'))
plt.title(dogs_name_dict[np.argmax(valid_img[1][num])])
plt.show()

y_pred = model(np.expand_dims(valid_img[0][num],axis=0)).numpy()[0]
names = np.argsort(y_pred)[::-1][:4]

for name in names:
    print(y_pred[name],":",dogs_name_dict[name])

In [ ]:
#### pomeranian(포메라니안) Test 이미지로 오류 분석 하기 #######################################
## 흰색 포메라니안 이미지 예측
pomeranian_img_path = '/Users/yoonjeongyang/Desktop/AI-exer/augmentation/stanford_dogs/test_images/white.jpg'
pomeranian_img =cv2.imread(pomeranian_img_path)
pomeranian_img = cv2.cvtColor(pomeranian_img,cv2.COLOR_BGR2RGB)
pomeranian_img = cv2.resize(pomeranian_img,(224,224))
plt.imshow(pomeranian_img)
plt.axis('off')
plt.show()

y_pred = model(np.expand_dims(pomeranian_img,axis=0)).numpy()[0]
names = np.argsort(y_pred)[::-1][:4]

for name in names:
    print(y_pred[name],":",dogs_name_dict[name])

In [ ]:
## 갈색 포메라니안 이미지 예측 
pomeranian_img_path = '/Users/yoonjeongyang/Desktop/AI-exer/augmentation/stanford_dogs/test_images/brown.jpg'
pomeranian_img =cv2.imread(pomeranian_img_path)
pomeranian_img = cv2.cvtColor(pomeranian_img,cv2.COLOR_BGR2RGB)
pomeranian_img = cv2.resize(pomeranian_img,(224,224))
plt.imshow(pomeranian_img)
plt.axis('off')
plt.show()

y_pred = model(np.expand_dims(pomeranian_img,axis=0)).numpy()[0]
names = np.argsort(y_pred)[::-1][:4]

for name in names:
    print(y_pred[name],":",dogs_name_dict[name])

In [ ]:
#### 포메라니안 학습 이미지 확인하기 ############################################################
pomeranian_path = '/Users/yoonjeongyang/Desktop/AI-exer/augmentation/stanford_dogs/Images/Pomeranian/'
pomeranian_list = os.listdir(pomeranian_path)

plt.figure(figsize=(30,20))
np.random.shuffle(pomeranian_list) # 이미지들의 순서를 무작위로 섞어 골고루 이미지를 뽑아 올 수 있도록 한다.
for i in range(60):
    plt.subplot(6,10,i+1)
    img =cv2.imread(pomeranian_path+pomeranian_list[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(100,100))
    plt.imshow(img)
    plt.axis("off")
plt.show()

In [ ]:
#### Dandie Terrier 이미지의 클래스 활성화 맵 분석 ##################################################
pomeranian_img_path = '/Users/yoonjeongyang/Desktop/AI-exer/augmentation/stanford_dogs/test_images/dandie_terrier.jpg'
pomeranian_img =cv2.imread(pomeranian_img_path)
pomeranian_img = cv2.cvtColor(pomeranian_img,cv2.COLOR_BGR2RGB)
pomeranian_img = cv2.resize(pomeranian_img,(224,224))

In [ ]:
img = pomeranian_img.copy()
img = np.expand_dims(img,[0])
img = tf.Variable(img)

In [ ]:
preds = model.predict(img.numpy())[0]

In [ ]:
temp_conv_layer = model.get_layer('activation_10')

In [ ]:
temp_input_model = models.Model(model.input, temp_conv_layer.input)
temp_output_model = models.Model(temp_conv_layer.output, model.output)

In [ ]:
with tf.GradientTape() as tape:
    temp_input_model_ouput = temp_input_model(img)
    temp_input_model_ouput = tf.Variable(temp_input_model_ouput)
    temp_output_model_output = temp_output_model(temp_input_model_ouput)[0,np.argmax(preds)]

In [ ]:
grads = tape.gradient(temp_output_model_output,temp_input_model_ouput)

pooled_grads = tf.reduce_mean(grads,axis=(0,1,2))

pooled_grads_value = pooled_grads.numpy()

In [ ]:
conv_layer_output_value = temp_input_model_ouput[0].numpy()

In [ ]:
for i in range(512):
    conv_layer_output_value[:,:,i] *= pooled_grads[i]

In [ ]:
heatmap = np.mean(conv_layer_output_value,axis=-1)

In [ ]:
plt.imshow(heatmap)
plt.show()

In [ ]:
heatmap = cv2.resize(heatmap, (pomeranian_img.shape[1],pomeranian_img.shape[0]))
heatmap = np.abs(heatmap)

In [ ]:
heatmap_rgb = (heatmap - np.min(heatmap))/(np.max(heatmap) - np.min(heatmap))
# minmax 정규화 -> 0~1로 정규화
heatmap_rgb = np.uint8(heatmap_rgb*255) # RGB포맷(0~255)으로 변경

In [ ]:
heatmap = cv2.applyColorMap(heatmap_rgb, cv2.COLORMAP_JET)

In [ ]:
superimposed_img = heatmap*0.4 + pomeranian_img
superimposed_img = np.minimum(255,superimposed_img).astype('uint8')

In [ ]:
plt.imshow(np.minimum(255,superimposed_img).astype('uint8'))
plt.title("Dandie_terrier")
plt.colorbar()
plt.show()

In [ ]:
names = np.argsort(preds)[::-1][:4]

for name in names:
    print(preds[name],":",dogs_name_dict[name])

In [ ]:
#### Top3 클래스 정확도 확인 #####################################################################
y_pred_top3 = np.argsort(model(valid_img[0]))[:,::-1][:,:3]
y = np.argmax(valid_img[1],axis=1)

accuracy = 0
for i in range(len(y)):
    if(np.sum(y_pred_top3[i] == y[i])):
        accuracy +=1

accuracy /=240
print(accuracy)